In [ ]:
pip install dune-client

In [ ]:
from dotenv import load_dotenv

load_dotenv(".env")

In [ ]:
import os
from dune_client.client import DuneClient

DUNE_API_KEY=os.getenv("DUNE_API_KEY")
dune = DuneClient(DUNE_API_KEY)
query_result = dune.get_latest_result(3888983)
print(query_result.result)

In [ ]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase

In [ ]:
query = QueryBase(
    name="ethereum_get_info_from_symbol",
    query_id=3893389,
    params=[
        QueryParameter.text_type(name="symbol", value="usdt"),
    ],
)
results_df = dune.run_query_dataframe(query)
print(results_df.to_json())

In [ ]:
from web3 import Web3

infura_url = f"https://mainnet.infura.io/v3/{os.getenv('INFURA_API_KEY')}"
web3 = Web3(Web3.HTTPProvider(infura_url))

def is_contract_address(address):
    code = web3.eth.get_code(Web3.to_checksum_address(address))
    return code != b''

print(is_contract_address("0xC0ffeEBABE5D496B2DDE509f9fa189C25cF29671"))

In [ ]:
from dune_tools import top_10_eth_transaction

result=top_10_eth_transaction.invoke({"min_amout":1000})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address
import json

token_balances_str = get_token_balances_of_address.invoke(
    {"address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d"}
)
token_balances = json.loads(token_balances_str)
    # 生成Token余额分析报表
token_balance_report = []
for token in token_balances:
    balance_usd = token.get("balance_usd", "N/A")
    token_balance_report.append(
        f"Token: {token['token_symbol']} (Contract: {token['token_address']}, Liquidity on DEX in 3 months:${token['total_liquidity_usd']})\n"
        f"Balance: {token['balance']} (USD: {balance_usd} Price:${token['price']})\n"
    )

# Token余额分析部分
token_analysis = "\n".join(token_balance_report)
print(token_analysis)

In [ ]:
import requests
import time
import os

def get_address_tags_from_etherscan(addresses):
    ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")  # Replace with your actual Etherscan API key
    ETHERSCAN_API_URL = 'https://api.etherscan.io/api'
    
    address_tags = {}
    
    for address in addresses:
        params = {
            'module': 'account',
            'action': 'txlist',
            'address': address,
            'apikey': ETHERSCAN_API_KEY,
            'startblock': 0,
            'endblock': 99999999,
            'page': 1,
            'offset': 10,  # Example to limit query, adjust as necessary
            'sort': 'asc'
        }
        
        response = requests.get(ETHERSCAN_API_URL, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data['status'] == '1':
                # Extracting labels from the result
                address_tags[address] = data['result']
            else:
                address_tags[address] = 'No data found'
        else:
            address_tags[address] = f"Error: {response.status_code}"
        
        # Respect Etherscan API rate limits
        time.sleep(1)
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e", 
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D"
]
address_tags = get_address_tags_from_etherscan(addresses)
for address, tags in address_tags.items():
    print(f"Address: {address}, Tags: {tags}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup

def get_address_tag_from_etherscan(address):
    """
    Get the tag of a single Ethereum address from Etherscan.
    """
    url = f"https://etherscan.io/address/{address}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        tag_element = soup.find("span", {"class": "u-label u-label--xs u-label--info rounded"})

        if tag_element:
            return tag_element.text.strip()
        else:
            return "No tag found"
    else:
        return f"Failed to retrieve page, status code: {response.status_code}"

def get_multiple_address_tags(addresses):
    """
    Get tags for multiple Ethereum addresses from Etherscan.
    """
    address_tags = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(get_address_tag_from_etherscan, addresses)
        
    for address, tag in zip(addresses, results):
        address_tags[address] = tag
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e",
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D",
    "0x5a52E96BACdAbDcf4b16a7F8cE7C6c31E3Cc4aF6"
]

address_tags = get_multiple_address_tags(addresses)
for address, tag in address_tags.items():
    print(f"Address: {address}, Tag: {tag}")

In [ ]:
from dune_tools import get_address_tag_from_etherscan

result=get_address_tag_from_etherscan.invoke({"address":"0xF838ac8921e8d4Efdd20e605C2855f20691ba6a3"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0xA0c68C638235ee32657e8f720a23ceC1bFc77C77"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0x59a19d8c652fa0284f44113d0ff9aba70bd46fb4"})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address,extract_main_token_of_balances

# balances=get_token_balances_of_address.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
# print(balances)
# result=extract_main_token_of_balances(balances)
# print(result)
from dune_tools import extract_token_from_balances
print(extract_token_from_balances(data=balances))

In [ ]:
from dune_tools import get_erc20_transfer_of

result=get_erc20_transfer_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d','contract_address':'0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'})
print(result)

In [ ]:
from dune_tools import get_eth_movements_of

result=get_eth_movements_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
print(result)

In [ ]:
import pprint
from ethereum_address_analysis_agent import ethereum_address_analysis_agent_executor

# ethereum_address_analysis_agent_executor.invoke(
#     {
#         "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
#         "chat_history": [],
#     },
#     config={
#         "configurable": {"llm": "openai_gpt_4o"},
#     },
# )

chunks = []
async for chunk in ethereum_address_analysis_agent_executor.astream_events(
    {
        "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "chat_history": [],
    },
    version="v1",
    config={
        "configurable": {"llm": "openai_gpt_4o"},
    },
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

In [1]:
from dune_tools import get_balances_of_address

result=get_balances_of_address.invoke({"address":"0xdfd5293d8e347dfe59e90efd55b2956a1343963d"})

2024-08-08 11:53:31,232 INFO botocore.credentials Found credentials in shared credentials file: ~/.aws/credentials
2024-08-08 11:53:31,451 INFO dune_client.api.base executing 3916915 on medium cluster
2024-08-08 11:53:32,866 INFO dune_client.api.base waiting for query execution 01J4R1XN3A37MJQ3A31G1B2B7W to complete: ExecutionState.PENDING (queue position: 1)
2024-08-08 11:53:34,121 INFO dune_client.api.base waiting for query execution 01J4R1XN3A37MJQ3A31G1B2B7W to complete: ExecutionState.EXECUTING
2024-08-08 11:53:35,389 INFO dune_client.api.base waiting for query execution 01J4R1XN3A37MJQ3A31G1B2B7W to complete: ExecutionState.EXECUTING
2024-08-08 11:53:36,680 INFO dune_client.api.base waiting for query execution 01J4R1XN3A37MJQ3A31G1B2B7W to complete: ExecutionState.EXECUTING
2024-08-08 11:53:37,939 INFO dune_client.api.base waiting for query execution 01J4R1XN3A37MJQ3A31G1B2B7W to complete: ExecutionState.EXECUTING
2024-08-08 11:53:39,260 INFO dune_client.api.base waiting for quer

In [ ]:
pip install matplotlib

In [ ]:
from dune_tools import get_token_balance_daily_of_address

result=get_token_balance_daily_of_address.invoke({"address":"0xdfd5293d8e347dfe59e90efd55b2956a1343963d","token_address":"0x4575f41308ec1483f3d399aa9a2826d74da13deb"})
print(result)

In [7]:
from tools_basic import tools
import json

t_arr=[{
            "name": tool.name,
            "description": tool.description,
            "parameters": {
                "type": "string",  # 假设工具接收的参数类型是字符串
                "format": f"{tool.name}({{{tool.name}_input}})"  # 定义参数格式
            }
        } for tool in tools]

print(json.dumps(t_arr))

[{"name": "what_is_time_now", "description": "Useful when you need to get the current time.", "parameters": {"type": "string", "format": "what_is_time_now({what_is_time_now_input})"}}, {"name": "get_address_labels", "description": "Useful when you need to get a label for some addresses.", "parameters": {"type": "string", "format": "get_address_labels({get_address_labels_input})"}}, {"name": "get_balances_of_address", "description": "Useful when you need to get the balance of all tokens on an address and an image about token balance distribution in USD.", "parameters": {"type": "string", "format": "get_balances_of_address({get_balances_of_address_input})"}}, {"name": "get_token_balance_daily_of_address", "description": "This is useful when you need to get daily balance data for a specified token on an address within 1 months.", "parameters": {"type": "string", "format": "get_token_balance_daily_of_address({get_token_balance_daily_of_address_input})"}}, {"name": "top_10_erc20_transaction

In [1]:
from dune_tools import get_address_labels

result=get_address_labels.invoke({"addresses":["0xdfd5293d8e347dfe59e90efd55b2956a1343963d"]})
print(result)

/Users/zhangleping/github.com/holynull/ethereum_address_analysis_agent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-02 09:56:00,136 INFO dune_client.api.base executing 3912735 on medium cluster
2024-08-02 09:56:01,561 INFO dune_client.api.base waiting for query execution 01J48CT6Z7E4H3QMKSXHDNZ6KA to complete: ExecutionState.PENDING (queue position: 1)
2024-08-02 09:56:02,901 INFO dune_client.api.base waiting for query execution 01J48CT6Z7E4H3QMKSXHDNZ6KA to complete: ExecutionState.EXECUTING
2024-08-02 09:56:04,241 INFO dune_client.api.base waiting for query execution 01J48CT6Z7E4H3QMKSXHDNZ6KA to complete: ExecutionState.EXECUTING
/Users/zhangleping/github.com/holynull/ethereum_address_analysis_agent/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainD

[{"address":"0xdfd5293d8e347dfe59e90efd55b2956a1343963d","labels":"[Uniswap User $5k-$10k avg. DEX trade value 2 years old DEX trader Number of DEXs traded on: 1 DEX Trader Sparse Trader Binance User Binance 16][Binance 16]"}]


In [1]:
from dune_tools import get_addres_how_much_funds_transfered

result = get_addres_how_much_funds_transfered.invoke(
    {
        "address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "contract_address": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
        "token_sybmol": "USDC",
        "sta_time": "2024-07-29 00:00:00",
        "end_time": "2024-08-06 23:59:59",
    }
)
print(result)

ModuleNotFoundError: No module named 'dotenv'